In [ ]:
import numpy as np
import sys
from scipy.optimize import least_squares, fmin
from plotly.subplots import make_subplots
import plotly.graph_objects as go
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.tire_model.tire_model_utils import *
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car, to_vel_frame, to_car_frame

In [ ]:
ARB_SETTINGS = ["Disconnected", "Soft-Soft", "Soft-Medium", "Medium-Medium", "Medium-Hard", "Hard-Hard"]
def find_roll_stiffness(front_arb, rear_arb, car: Car):
    front_bellcrank_ratios = np.array([0.0, 0.28843, 0.32397, 0.35952, 0.38328, 0.40705])
    rear_bellcrank_ratios = np.array([0.0, 0.26927, 0.31020, 0.35112, 0.39029, 0.42945])
    front_bellcrank_ratio = front_bellcrank_ratios[ARB_SETTINGS.index(front_arb)]
    rear_bellcrank_ratio = rear_bellcrank_ratios[ARB_SETTINGS.index(rear_arb)]
    front_arb_rate, rear_arb_rate = 3.39 * FTLB_TO_NM, 7.86 * FTLB_TO_NM
    lever_arm_length = 2.5 * IN_TO_M
    front_arb_setting = front_arb_rate * (front_bellcrank_ratio**2) * (car.front_track**2) / (lever_arm_length**2)
    rear_arb_setting = rear_arb_rate * (rear_bellcrank_ratio**2) * (car.rear_track**2) / (lever_arm_length**2)
    front_ride_roll_stiffness, rear_ride_roll_stiffness = 172 * FTLB_TO_NM, 172 * FTLB_TO_NM
    front_roll_stiffness, rear_roll_stiffness = front_arb_setting + front_ride_roll_stiffness, rear_arb_setting + rear_ride_roll_stiffness
    return front_roll_stiffness, rear_roll_stiffness

In [ ]:
skidpad_radius_inner = 15.25 / 2 # m
skidpad_radius_outer = skidpad_radius_inner + 3 # m
skidpad_radius = skidpad_radius_inner + 0.711 # m
print(skidpad_radius)


In [ ]:
# generate a car object
car = Car(mass=(480+150)*LB_TO_KG, front_axle_weight=0.48)
car.set_tire(H_R20_18X6_7)
car.mass_unsprung = 60 * LB_TO_KG
frs, rrs = find_roll_stiffness("Medium-Hard", "Medium-Hard", car)
car.k_f = frs
car.k_r = rrs
car.k_c = 1400 * LB_TO_KG
car.cg_height = 11.6 * IN_TO_M
car.z_f = -1.14 * IN_TO_M
car.z_r = -1.02 * IN_TO_M
car.toe_front = -0.15
car.toe_rear = -0.15
car.update_car()

In [ ]:
# converge on the minimum lap time of the skidpad
kappa_lim = 0.2
beta, delta, ay, yaw, ax, bruh, vel, lap_time = car.solve_skidpad_time(skidpad_radius, 0.65, sr_lim = kappa_lim)

print('Velocity: {:.2f} m/s'.format(vel))
print('Beta: {:.2f} deg'.format(np.rad2deg(beta)))
print('Delta: {:.2f} deg'.format(np.rad2deg(delta)))
print('ay: {:.2f} m/s^2'.format(ay))
print('yaw: {:.2f} rad/s^2'.format(yaw))
print('ax: {:.2f} m/s^2'.format(ax))
print(f'LLTD: {car.LLTD:.3f}')
print('bruh: {}'.format(bruh))
print('Lap time: {:.4f} s'.format(lap_time))
# omega based on lateral acceleration
omega = vel / skidpad_radius
print('Omega: {:.2f} rad/s'.format(omega))



In [ ]:
# make a plot that shows the force and torque vectors at each tire, the car's center of mass, and the center of the skidpad
fig = go.Figure()
# add the skidpad as a circle
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius_inner, y0=-skidpad_radius_inner, x1=skidpad_radius_inner, y1=skidpad_radius_inner, line_color="Red", line_width=1)
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius, y0=-skidpad_radius, x1=skidpad_radius, y1=skidpad_radius, line_color="LightSeaGreen", line_width=2)
fig.add_shape(type="circle", xref="x", yref="y", x0=-skidpad_radius_outer, y0=-skidpad_radius_outer, x1=skidpad_radius_outer, y1=skidpad_radius_outer, line_color="Red", line_width=1)
# add a point at the center of the skidpad
fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker=dict(color='LightSeaGreen', size=5), showlegend=False))
# add the car's center of mass
fig.add_trace(go.Scatter(x=[-skidpad_radius], y=[0], mode='markers', marker=dict(color='red', size=5), showlegend=False))
# find coordinates of the tires
tire_xy_car = np.array([[-car.front_track / 2, car.a], [car.front_track / 2, car.a], [-car.rear_track / 2, -car.b], [car.rear_track / 2, -car.b], [0, 0]])
# rotate the tire coordinates to the car's orientation
rot = np.array([[np.cos(-beta), -np.sin(-beta)], [np.sin(-beta), np.cos(-beta)]]) # were rotating it back to world coordinates so we need to negate the angle
tire_xy_vel = np.matmul(rot, tire_xy_car.T).T
# shift the tire coordinates to the car's position
tire_xy_vel[:, 0] -= skidpad_radius
# add the tire coordinates to the plot
# fig.add_trace(go.Scatter(x=tire_xy_vel[:, 0], y=tire_xy_vel[:, 1], mode='markers', marker=dict(color='blue', size=2), showlegend=False))
# get the force and torque vectors at each tire
ay_v, yaw_it, ax_v, bruh, fy_s, fx_s, mz_s, fz_s, sa_s, delta_s, kappa_s = car.solve_for_yaw(ay, vel, beta, delta, mu_corr=0.65, vecs=True, sr_lim = kappa_lim)
scalar = 800
# add the force and torque vectors to the plot
names = ['FL', 'FR', 'RL', 'RR', 'CM']
def rot_vec(point, rot):
    pt = np.matmul(rot, point.T).T
    return np.array([[0, 0], pt])
for i in range(5):
    t_rot = np.array([[np.cos(-delta_s[i]), -np.sin(-delta_s[i])], [np.sin(-delta_s[i]), np.cos(-delta_s[i])]])
    f_tot = rot_vec(np.array([-fy_s[i] / scalar, fx_s[i] / scalar]), t_rot)
    f_x = rot_vec(np.array([0, fx_s[i] / scalar]), t_rot)
    f_y = rot_vec(np.array([-fy_s[i] / scalar, 0]), t_rot)
    f_tot = np.matmul(rot, f_tot.T).T + tire_xy_vel[i]
    f_x = np.matmul(rot, f_x.T).T + tire_xy_vel[i]
    f_y = np.matmul(rot, f_y.T).T + tire_xy_vel[i]
    name = names[i]
    fig.add_trace(go.Scatter(x=f_tot[:, 0], y=f_tot[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='green', width=2), hovertext='{}: {:.2f} N'.format(name, np.sqrt(fx_s[i]**2 + fy_s[i]**2))))
    fig.add_trace(go.Scatter(x=f_x[:, 0], y=f_x[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='pink', width=2), hovertext='{} Fx: {:.2f} N'.format(name, fx_s[i]), showlegend=False))
    fig.add_trace(go.Scatter(x=f_y[:, 0], y=f_y[:, 1], legendgroup=f"{name}", name=name, mode='lines', line=dict(color='pink', width=2), hovertext='{} Fy: {:.2f} N'.format(name, fy_s[i]), showlegend=False))
# add the car's orientation
# fig.add_trace(go.Scatter(x=[skidpad_radius, skidpad_radius + np.sin(beta)], y=[0, np.cos(beta)], mode='lines', line=dict(color='red', width=2)))
print(f"Fz: {fz_s}")
print(f"SA: {np.rad2deg(np.array(sa_s))}")
print(f"Delta: {np.rad2deg(np.array(delta_s))}")
print(f"SL: {kappa_s}")
print(ay_v, ax_v)

fig.update_layout(title='Skidpad Forces and Torques', xaxis_title='x (m)', yaxis_title='y (m)', width=1000, height=1000, template="plotly_dark")
fig.update_xaxes(scaleanchor="y", scaleratio=1)
fig.show()


In [ ]:
ax_c, ay_c = to_car_frame(ax, ay, beta)
fzfl, fzfr, fzrl, fzrr, wt_pitch, wt_roll = car.find_contact_patch_loads(ax_c, ay_c, vel)
print(fzfl, fzfr, fzrl, fzrr, wt_pitch, wt_roll)